In [ ]:
# importing libraries
import pandas as pd              # for data manipulation
import numpy as np               # for mathematical operations

The "deliveries.csv" file stores the ball by ball information of the Indian Premier League from 2008 to 2019.

In [ ]:
# reading the "deliveries" dataset
deli = pd.read_csv("deliveries.csv")

# displaying the dataframe
deli.head()

The "cricket.xls" file stores the performance details and auction prices in IPL 2011 of 129 cricket players. 

In [ ]:
# reading the "cricket" dataset
df = pd.read_excel("cricket.xls")

# displaying the dataframe
df.tail()

### Using the 2 dataframes, we will create one resultant dataframe which stores the auction prices, general performance details, and performance details, wherever available, in IPL for the 129 cricket players. 

The last few rows of the "cricket" dataframe do not store any information and hence need to be removed.

In [ ]:
# filtering out the unnecessary rows from the dataframe
df = df[df.index <129]

In [ ]:
# displaying the dataframe
df.tail()

Since the auction prices are available for the year 2011, the IPL data will be studied for 2010, i.e., one year prior to 2011. 

The "matches.csv" file is read to find out the match ids of the games that took place in IPL 2010.

In [ ]:
# reading the "matches" dataframe
matches = pd.read_csv("matches.csv")

# displaying the dataframe
matches.head()

Using the column "season" and specifying its value to 2010, the dataframe is filtered out and the match ids are extracted.

In [ ]:
# filtering out the dataframe and extracting the match ids
match_id = matches[matches.season == 2010].id.values

In [ ]:
# 60 matches took place in IPL 2010
len(match_id)

Next, the deliveries that took place in IPL 2010 are filtered using the match ids that was created in the previous step. 

In [ ]:
# filtering the dataframe for deliveries of IPL 2010
deli = deli[deli.match_id.isin(match_id)]

# displaying the dataframe
deli.head()

The names of all the cricket players are extracted from the "cricket" dataframe and stored.

In [ ]:
# extracting the names
players = df.Name

# displaying the names
players

It is to be noted that the naming conventions used in the two dataframes are different. While the "cricket" dataset uses the full name, the "deliveries" dataset uses the full surname and initials of the first name. Hence, the surnames need to be used to link the records in the two dataframes. Again, in the "cricket" dataset, the players who were retained by their IPL teams have the string '(retained)' added at the end of their names, which needs to be removed. 



In [ ]:
# function to extract the surname
def get_surname(l):
    
    # empty dictionary to store the player names and their surnames
    names = {}
    
    # iterating over the player names
    for name in l:
        # extracting the last part of the string
        surname = name.split()[-1]
        # if the last element of the splitted string is '(retained)', select the element left to it
        if surname == '(retained)':
            surname = name.split()[-2]    
        # add surname to the dictionary
        names.update({name:{'surname':surname}})
        
    # return the dictionary
    return names

In [ ]:
# extract the surnames
player_names = get_surname(players)

# displaying the resultant dictionary
player_names

Now, the names of the players in the "deliveries" dataset are given in two separate columns: "batsman" and "bowler" since cricket players are either batsmen or bowlers or both. Hence, the names need to be extracted using the two columns.

First, the "batsman" column is used to find the corresponding names in the "deliveries" dataset.

Even then, multiple players can have the same surname. So, the first letter of their first name is extracted and checked in the available initials for that surname. If a match is found, that name is added as a key to that player. 

There are also many players who were bought at the auction but did not get a chance to play any game. For such players, the key is not added. 

In [ ]:
# function to get the names in the "deliveries" dataset 
def get_batsman_info(surname):
    
    # finding all the rows where the batsman name contains the passed surname
    data = deli[deli.batsman.str.contains(surname)]
    
    # returning all the unique batsman names with that specific surname
    return data.batsman.unique()

In [ ]:
# iterating over all the player names
for b in player_names.keys():
    
    # surname of the player
    surname = player_names[b]['surname']
    
    # list of all players who have the specific surname, can be 1 or more
    list_of_names = get_batsman_info(surname)
    
    # exception handling in case a match for the surname does not exist in the "deliveries" dataset
    try:
        # iterating over all the names in the list
        for n in list_of_names:
            # checking if the first letter of the first name is present in the initials
            if b[0] in n.split(' ')[0]:
                # adding the key 
                player_names[b]['key'] = n
    except:
        pass
            

In [ ]:
# displaying the dictionary
player_names

The corresponding keys for all batsmen who played in IPL 2010 have been added. The steps are repeated for the bowlers. 

In [ ]:
# function to get the names in the "deliveries" dataset 
def get_bowler_info(surname):
    
    # finding all the rows where the bowler name contains the passed surname
    data = deli[deli.bowler.str.contains(surname)]
    
    # returning all the unique bowler names with that specific surname
    return data.bowler.unique()

In [ ]:
# iterating over all the player names
for key in player_names.keys():
    
    # in case a key for the player name has not been added
    if 'key' not in player_names[key].keys():
        
        # surname of the player
        surname = player_names[key]['surname']
        
        # list of all players who have the specific surname, can be 1 or more
        list_of_names = get_bowler_info(surname)
        
        # exception handling in case a match for the surname does not exist in the "deliveries" dataset
        try:
            # iterating over all the names in the list
            for n in list_of_names:
                # checking if the first letter of the first name is present in the initials 
                if key[0] in n.split(' ')[0]:
                    # adding the key 
                    player_names[key]['key'] = n
        except:
            pass


In [ ]:
# printing the names of all players who did not play a single game in IPL 2010
for key in player_names.keys():
    # in case a key for the player name has not been added
    if 'key' not in player_names[key].keys():
        # print name
        print(key)

Two players did play games but their first names were stored as the middle names in the "deliveries" dataset. Their keys are added manually.

In [ ]:
player_names['Vinay Kumar']['key'] = 'R Vinay Kumar'
player_names['Venugopal Rao']['key'] = 'Y Venugopal Rao'

In [ ]:
# displaying the final dictionary
player_names

Now, the mapping between the names of the players in the two dataframes has been successfully created. The next step is to get the batting and details for the players in IPL 2010.

In [ ]:
# function to get the batting details given the player name
def get_batting_details(name):
    
    # filtering out the deliveries faced by the batsman
    data = deli[deli.batsman == name]
    # no. of matches in which the batsman batted
    num_matches = len(data.match_id.unique())
    # total runs scored by the batsman in the entire season
    total_runs = data.batsman_runs.sum()
    # balls faced by the batsman in the entire season
    balls_faced = data.ball.count()
    # runs scored per match by the batsman
    runs_per_match = data.groupby('match_id').sum()['batsman_runs']
    # balls faced per match by the batsman
    balls_faced_per_match = data.groupby('match_id').count()['ball']
    # batting average in the season
    batting_avg = np.mean(runs_per_match)
    # highest score in the season, 0 in case the player did not score in any match
    highest_score = max(runs_per_match, default=0)
    # average strike rate in the season
    strike_rate = np.mean(runs_per_match/balls_faced_per_match) * 100
    # number of centuries scored in the season
    num_centuries = len([runs for runs in runs_per_match if runs>=100])
    # number of half centuries scored in the season
    num_half_centuries = len([runs for runs in runs_per_match if runs>=50])
    # number of fours hit in the season
    fours = len(data[data.batsman_runs == 4])
    # number of sixes hit in the season
    sixes = len(data[data.batsman_runs == 6])
    
    # making a dictionary 
    tmp = {'matches_bat_ipl':num_matches, 'total_runs_scored_ipl':total_runs, 'balls_faced_ipl':balls_faced,
          'batting_avg_ipl':batting_avg, 'highest_score_ipl':highest_score, 'strike_rate_ipl':strike_rate,
          'centuries_ipl':num_centuries, 'half_centuries_ipl':num_half_centuries, 'fours_ipl':fours,
          'sixes_ipl':sixes}
    
    # returning the dictionary
    return tmp

In [ ]:
# function to get the bowling details given the player name
def get_bowling_details(name):
    
    # filtering out the deliveries bowled by the bowler
    data = deli[deli.bowler == name]
    # no. of matches in which the bowler bowled
    num_matches = len(data.match_id.unique())
    # total runs given by the bowler in the entire season
    total_runs = data.batsman_runs.sum()
    # balls bowled by the bowler in the entire season
    balls_bowled = data.ball.count()
    # wickets taken by the bowler in the entire season
    wickets = len(data[data.player_dismissed.notnull()])
    # runs given per match by the bowler
    runs_per_match = data.groupby('match_id').sum()['batsman_runs']
    # balls bowled per match by the bowler
    balls_bowled_per_match = data.groupby('match_id').count()['ball']
    # average runs given per match in the season
    bowling_avg = np.mean(runs_per_match)
    # average economy in the season
    economy = np.mean(runs_per_match/balls_bowled_per_match*6)
    # wickets taken per match
    wickets_per_match = data[data.player_dismissed.notnull()].groupby('match_id').count()['player_dismissed']
    # no. of 3 wicket hauls in the season
    three_wickets = len(wickets_per_match[wickets_per_match >= 3])
    
    # making a dictionary 
    tmp = {'matches_bowl_ipl':num_matches, 'balls_bowled_ipl':balls_bowled, 'total_runs_given_ipl':total_runs,
          'wickets_taken_ipl':wickets, 'bowling_avg_ipl':bowling_avg, 'economy_ipl':economy, '3wickets_ipl':three_wickets}
    
    # returning the dictionary
    return tmp

Now, the performance data for the players in IPL 2010 will be collected.

In [ ]:
# initialise an empty dictionary
final = {}

In [ ]:
# iterating over the player names
for key in player_names.keys():
    # exception handling in case a key for the player name does not exist
    try:
        # name of the player in the "deliveries" dataset
        name = player_names[key]['key']
        # getting the batting information for the player
        result = get_batting_details(name)
        # getting the bowling information for the player
        result.update(get_bowling_details(name))
        # updating the final dictionary with the name as the key
        final.update({key:result})
    except:
        pass

In [ ]:
# creating a dataframe using the dictionary
ipl_df = pd.DataFrame(final).T

ipl_df.head()

In [ ]:
# reset index of the dataframe
ipl_df.reset_index(inplace=True)

In [ ]:
# changing the column name
ipl_df.rename(columns={'index':'Name'}, inplace=True)

In [ ]:
# display the dataframe
ipl_df.head()

The required dataframe storing the information on the performance of the players in IPL 2010 has been created. 

In the "cricket" dataset, create a categorical column "Retained" which stores 1 if the player was retained by his team, otherwise 0.

In [ ]:
# list storing information for the "Retained" variable
retained = []

# iterate over all the player names
for name in df['Name']:
    
    # split the name; if the last element is '(retained)', store 1, else 0
    if name.split()[-1] == '(retained)':
        retained.append(1)    
    else:
        retained.append(0)

In [ ]:
# adding the "Retained" column
df['Retained'] = retained

# update the "Name" column with the cleaned names
df['Name'] = player_names

In [ ]:
df

In [ ]:
# merging the two dataframes
final_data = df.merge(ipl_df, on='Name', how='outer')

In [ ]:
# displaying the info of the resultant dataframe
final_data.info()

In [ ]:
# saving the dataset
final_data.to_csv('Raw_Data.csv')

The dataset necessary for our study has been created, however, it is dirty. So the next step is to clean the dataframe.